In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
import matplotlib.pyplot as plt
from sksurv.metrics import concordance_index_censored as concordance
from sklearn.model_selection import StratifiedKFold as KF


In [ ]:
import glmnet_python

In [ ]:
from glmnet import glmnet

In [ ]:
import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
from glmnet import glmnet; from glmnetPlot import glmnetPlot
from glmnetPrint import glmnetPrint; from glmnetCoef import glmnetCoef; from glmnetPredict import glmnetPredict
from cvglmnet import cvglmnet; from cvglmnetCoef import cvglmnetCoef
from cvglmnetPlot import cvglmnetPlot; from cvglmnetPredict import cvglmnetPredict


In [ ]:
from sklearn.metrics import r2_score

In [ ]:
import timeit
import csv
import time

In [ ]:
Xdf = pd.read_csv('processedExpData.csv',index_col=0)
y = pd.read_csv('yData.csv',index_col=0)
cancer = pd.read_csv('cancerData.csv',index_col=0)

In [ ]:



def glmtestv2(lbdas=[1,0.5,0.2,0.1,0.05,0.02,0.01,0.005,0.002,0.001] ,outloop=5,inloop=5, l1_ratio=1):
    i=0
    np.random.seed(42)
    t0=time.time()
    index_map = pd.DataFrame(range(len(Xdf)),index=Xdf.index)

    outres_train = {}
    outres = {}
    inner_best = {}
    inner_best_train={}
    inner_var = {}
    errors = {}
    preds={}
    weights={}
    timeO = pd.DataFrame()
    
    for out in range(outloop):
        
        t1=time.time()

        df_scl = pd.DataFrame(StandardScaler().fit_transform(Xdf), index=Xdf.index,columns=Xdf.columns)
        totres_train = pd.DataFrame()
        totres = pd.DataFrame()
        totres_var = pd.DataFrame()
        idx=[]
        for cnc in list(cancer):
            np.random.seed(out)
            for st in [0,1]:

                idx.extend(index_map.loc[ np.random.choice(y[(y['vital_status']==st) & (cancer[cnc]==1)].index, 
                                        size=int(0.8*len(y[(y['vital_status']==st) & (cancer[cnc]==1)])),replace=False)][0].tolist())


        idx = np.sort(np.ravel(np.array(idx)))


        SS = StandardScaler()
        X_train = df_scl.iloc[idx,:]
        X_train = pd.DataFrame(SS.fit_transform(X_train),columns=X_train.columns,index=X_train.index)
        X_test = df_scl.iloc[[x for x in range(len(Xdf)) if x not in idx],:]
        X_test  = pd.DataFrame(SS.fit_transform(X_test),index=X_test.index,columns=X_test.columns)
        y_train = y.iloc[idx,:]
        y_test = y.iloc[[x for x in range(len(Xdf)) if x not in idx],:]
        can_train = cancer.iloc[idx,:]
        can_test = cancer.iloc[[x for x in range(len(Xdf)) if x not in idx],:]


        t1=time.time()
        results = {}
        trainresults = {}
        index_map2 = pd.DataFrame(range(len(X_train)),index=X_train.index)
        for b in range(inloop):
            t2=time.time()
            
            

            idx2 = []
            for cnc in list(cancer):
                np.random.seed(b)
                for st in [0,1]:

                    idx2.extend(index_map2.loc[ np.random.choice(y_train[(y_train['vital_status']==st) & (can_train[cnc]==1)].index, 
                                            size=int(0.75*len(y_train[(y_train['vital_status']==st) & (can_train[cnc]==1)])),replace=False)][0].tolist())


            idx2 = np.sort(np.ravel(np.array(idx2)))
            SS= StandardScaler()
            X_train2 = X_train.iloc[idx2,:]
            X_train2 = pd.DataFrame(SS.fit_transform(X_train2),columns=X_train2.columns,index=X_train2.index)
            X_test2 = X_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]
            X_test2 = pd.DataFrame(SS.fit_transform(X_test2),columns=X_test2.columns,index=X_test2.index)
            y_train2 = y_train.iloc[idx2,:]
            y_test2 = y_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]
            can_train2 = can_train.iloc[idx2,:]
            can_test2 = can_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]


            resdict = {}
            trainresdict = {}
            for a in list(cancer):
                
                print(a)
                #SS = StandardScaler()
                fit = glmnet(x = np.array(X_train2[can_train2[a]==1],dtype=scipy.float64), 
                         y = np.array(y_train2[can_train2[a]==1][['time','vital_status']].copy(),dtype=scipy.float64), 
                         family = 'cox',alpha=l1_ratio,lambdau=np.array(lamls[a]))
                trainresdict[a] = glmnetPredict(fit, np.array(X_train2[can_train2[a]==1].copy(),dtype=scipy.float64))

                resdict[a] = glmnetPredict(fit, np.array(X_test2[can_test2[a]==1].copy(),dtype=scipy.float64))

                
            res = pd.DataFrame()
            trainres = pd.DataFrame()
            for nl in range(len(lamls)):
                for a in list(cancer):

                    try:

                        trainres.loc[a,nl]= concordance(y_train2['vital_status'][can_train2[a]==1].astype('bool'),y_train2['time'][can_train2[a]==1],trainresdict[a][:,nl])[0]
                    except:

                        trainres.loc[a,nl] = np.nan

                    try:
                        res.loc[a,nl]= concordance(y_test2['vital_status'][can_test2[a]==1].astype('bool'),y_test2['time'][can_test2[a]==1],resdict[a][:,nl])[0]

                    except:
                        res.loc[a,nl] = np.nan


            print(i,time.time()-t0)
            results[b] = res
            trainresults[b] = trainres
            i+=1

        
        tmp = results[0]
        tmptrn = trainresults[0]
        for a in range(inloop-1):
            tmp+=results[a+1]
            tmptrn+=trainresults[a+1]
        
        tmp = tmp/inloop
        tmptrn = tmptrn/inloop
        
        totres = pd.DataFrame(tmp,columns=res.columns,index=res.index)
        totres_train = pd.DataFrame(tmptrn,columns=res.columns,index=res.index)
        
    
        print(totres.mean().idxmax())
#         bestparams = dict(totres.mean().idxmax())
#         bestparams['iter'] = out
        
        inner_best[out] = totres
        inner_best_train[out] = totres_train
       # inner_var[out] = totres_var
        outres[out] = pd.DataFrame()
        outres_train[out] = pd.DataFrame()

        weights[out] = {} 
        
        resdict = {}
        trainresdict = {}
        for a in list(cancer):
            
            tt=time.time()
            fit = glmnet(x = np.array(X_train[can_train[a]==1].copy(),dtype=scipy.float64), 
                         y = np.array(y_train[can_train[a]==1][['time','vital_status']].copy(),dtype=scipy.float64), 
                         family = 'cox',alpha=l1_ratio,lambdau=np.array(lamls[a]))
            trainresdict[a] = glmnetPredict(fit, np.array(SS.transform(X_train[can_train[a]==1].copy()),dtype=scipy.float64))

            resdict[a] = glmnetPredict(fit, np.array(X_test[can_test[a]==1].copy(),dtype=scipy.float64))
            timeO.loc[a,out] = time.time()-tt
            #weights[out][a] = pd.DataFrame(fit['beta'],index=X_test.columns,columns=lbdas[:fit['beta'].shape[1]])
        res = pd.DataFrame()
        trainres = pd.DataFrame()
        for nl in range(len(lamls)):
            for a in list(cancer):

                try:

                    trainres.loc[a,nl]= concordance(y_train['vital_status'][can_train[a]==1].astype('bool'),y_train['time'][can_train[a]==1],trainresdict[a][:,nl])[0]
                except:

                    trainres.loc[a,nl] = np.nan

                try:
                    res.loc[a,nl]= concordance(y_test['vital_status'][can_test[a]==1].astype('bool'),y_test['time'][can_test[a]==1],resdict[a][:,nl])[0]

                except:
                    res.loc[a,nl] = np.nan

        print(i,time.time()-t0)
        outres[out] = res
        outres_train[out] = trainres
        i+=1


        
          
    return outres , outres_train,inner_best,inner_best_train,  timeO #,weights #preds,weights,

In [ ]:
vals = [float(np.format_float_positional(1 * 0.75**i, precision=2, unique=False, fractional=False, trim='k')) for i in range(33)]
vals

In [ ]:
glassoO1,glassoO2,glassoI1,glassoI2,glassot   = glmtestv2(lbdas=vals+[0] ,outloop=5,inloop=5,l1_ratio=1)

In [ ]:
gelnetO1,gelnetO2,gelnetI1,gelnetI2,gelnett = glmtestv2(lbdas=vals+[0] ,outloop=5,inloop=5,l1_ratio=0.5)

In [ ]:
glmnet_elnet_res = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(gelnetI1[b].index):
        
        print(a)
        glmnet_elnet_res.loc[a,b] =  gelnetO1[b].replace(np.nan,0.5)[gelnetI1[b].replace(np.nan,0.5).idxmax(1)[a]][a]
 

In [ ]:
glmnet_lasso_res = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(glassoI1[b].index):
        
        
        glmnet_lasso_res.loc[a,b] =  glassoO1[b].replace(np.nan,0.5)[glassoI1[b].replace(np.nan,0.5).idxmax(1)[a]][a]
 

In [ ]:
glmnet_elnet_res

In [ ]:
glmnet_lasso_res